In [1]:
import requests
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import re
from datetime import datetime
import pandas as pd
from connection_keys import connection, alchemy_connection

### Configurando a engine do banco

In [2]:
engine = alchemy_connection('db_hardware')

### Setup para abrir o navegador com o Selenium e puxar a primeira página

In [3]:
#--| Setup
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument('--disable-extensions')
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
 
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\igorS\.wdm\drivers\chromedriver\win32\87.0.4280.88]


In [4]:

# --| Parse or automation
browser.get('https://www.kabum.com.br/hardware/placa-de-video-vga?pagina=1')
html = soup(browser.page_source, 'html.parser')
# browser.implicitly_wait(5)

### Configuiração para percorer as páginas e salvar todos dados com a taga referente ao produto em uma lista

In [5]:
pagina = 0
html_produtos = []
flag = 0

while flag == 0:
    items = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
    if items != []:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.kabum.com.br/hardware/placa-de-video-vga?pagina={pagina}')
            html = soup(browser.page_source, 'html.parser')
            browser.implicitly_wait(5)
        except Exception: break
        html_produtos.append(items)
    else:
        break

### Tranformação da lista de listas em uma lista normal para percorer com mais facildiade os atributos do produto

In [6]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

### Percorrendo a lista, capurando e salvando cada atributo no dicionario

In [7]:
now = datetime.now()
produtos = []

for container in flat_list:
    
    # Estreutura de para salvar
    marca = container.div.img['alt']
    nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

    if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
        preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
    else: 
        preco_antigo_cartao = '0'
    preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
    preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
    image = container.find('div', {"class":"sc-fznKkj fEpBwY"})
    if "indisponivel" in image.img['src']:
        disponibilidade = 'Indisponivel'
    elif "disponivel" in image.img['src']:
        disponibilidade = 'Disponivel'

    produto = {
        "marca" : marca,
        "nome_produto": nome_produto,
        "preco_antigo_cartao": preco_antigo_cartao,
        "preco_atual_cartao": preco_atual_cartao,
        "preco_boleto": preco_boleto,
        "disponibilidade": disponibilidade,
        "data" : now.strftime("%Y-%m-%d"),
        "hora":  now.strftime("%H:%M:%S"),
        "loja": "Kabum" 
    }
    produtos.append(produto)


### Transformando o dict em Dataframe e visualizando uma amostra dos dados

In [8]:
df = pd.DataFrame(produtos)
df

,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,disponibilidade,data,hora,loja
0,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 165...,0,"1.529,29","1.299,90",Disponivel,2020-12-15,20:23:36,Kabum
1,Galax,Placa de Vídeo Galax NVIDIA GeForce RTX 2070 S...,0,"4.941,06","4.199,90",Disponivel,2020-12-15,20:23:36,Kabum
2,Sapphire,Placa de Vídeo Sapphire AMD Radeon RX 550 Puls...,0,"941,06","799,90",Disponivel,2020-12-15,20:23:36,Kabum
3,Gigabyte,Placa de Vídeo Gigabyte AMD Radeon RX 5600 XT ...,0,"3.058,71","2.599,90",Disponivel,2020-12-15,20:23:36,Kabum
4,Asus,Placa de Vídeo Asus TUF3 NVIDIA GeForce GTX 16...,0,"2.588,12","2.199,90",Disponivel,2020-12-15,20:23:36,Kabum
...,...,...,...,...,...,...,...,...,...
448,MSI,"Placa de Vídeo MSI Radeon RX 6800, 16G, GDDR6","6.235,18","5.882,24","4.999,90",Indisponivel,2020-12-15,20:23:36,Kabum
449,Gainward,Placa de Vídeo Gainward NVIDIA GeForce GTX 166...,"2.352,82","2.235,18","1.899,90",Indisponivel,2020-12-15,20:23:36,Kabum
450,Asus,"Placa de Vídeo Asus GeForce RTX 3060 Ti, 8GB -...",0,"5.882,24","4.999,90",Indisponivel,2020-12-15,20:23:36,Kabum
451,MSI,Placa de Vídeo MSI GeForce RTX 3060 Ti GAMING ...,"5.514,00","4.658,71","3.959,90",Indisponivel,2020-12-15,20:23:36,Kabum



### Criando a conexão com o banco e salvando os dados

In [21]:
df.to_sql('graphic_card', con=engine, if_exists='append', index = False)

----------------------------------------------------------------------------------------------------

(fora de uso devido a mudança nas politicas do site)

In [10]:
# #--| Parse or automation
# browser.get('https://www.pichau.com.br/hardware/placa-de-video?p=1')
# web = soup(browser.page_source, 'html.parser')
# browser.implicitly_wait(20)

In [11]:
# pagina = 0
# html_produtos = []
# flag = 0

# while pagina != 9:
#     items = web.findAll('div', {"class":"product-item-info"})
#     if items != []:
#         try:
#             pagina = pagina + 1
#             browser.get(f'https://www.pichau.com.br/hardware/placa-de-video?p={pagina}')
#             web = soup(browser.page_source, 'html.parser')
#         except Exception: break
#         html_produtos.append(items)
#     else:
#         break

In [12]:
# flat_list = []
# for sublist in html_produtos:
#     for item in sublist:
#         flat_list.append(item)

In [13]:
# def extract_brand(name):
#     if 'AFOX' in name.upper():
#         return 'AFOX'
#     elif 'AMD' in name.upper():
#         return 'AMD'
#     elif 'ASROCK' in name.upper():
#         return 'ASRock'
#     elif 'ASUS' in name.upper():
#         return 'Asus'
#     elif 'EVGA' in name.upper():
#         return 'EVGA'
#     elif 'GALAX' in name.upper():
#         return 'Galax'
#     elif 'GAINWARD' in name.upper():
#         return 'Gainward'
#     elif 'GIGABYTE' in name.upper():
#         return 'Gigabyte'
#     elif 'MSI' in name.upper():
#         return 'MSI'
#     elif 'PCYES!' in name.upper():
#         return 'PCYES'
#     elif 'PNY' in name.upper():
#         return 'PNY'
#     elif 'POWERCOLOR' in name.upper():
#         return 'PowerColor'
#     elif 'SAPPHIRE' in name.upper():
#         return 'Sapphire'
#     elif 'XFX' in name.upper():
#         return 'XFX'
#     elif 'ZOGIS' in name.upper():
#         return 'ZOGIS'
#     elif 'ZOTAC' in name.upper():
#         return 'Zotac'
#     else:
#         return ''

# now = datetime.now()
# produtos_pichau = []

# for container in flat_list:
#     if container.find('span', {"class":"price"}) != None:
#         # Estreutura de para salvar
#         marca = extract_brand(container.find('a', {"class":"product-item-link"}).text)
#         nome_produto = container.find('a', {"class":"product-item-link"}).text

#         preco_atual_cartao =  (container.find('span', {"class":"price"}).text)
#         preco_boleto = (container.find('span', {"class":"price-boleto"}).text).split(" ")[2]
        
#         disponibilidade = 'Disponivel'

#         produto = {
#             "marca" : marca,
#             "nome_produto": nome_produto,
#             "preco_antigo_cartao": '0',
#             "preco_atual_cartao": preco_atual_cartao,
#             "preco_boleto": preco_boleto,
#             "disponibilidade": disponibilidade,
#             "data" : now.strftime("%Y-%m-%d"),
#             "hora":  now.strftime("%H:%M:%S"),
#             "loja": "Pichau" 
#         }
#         produtos_pichau.append(produto)

In [14]:
# df_pichau = pd.DataFrame(produtos_pichau)
# df_pichau['preco_atual_cartao'] = df_pichau['preco_atual_cartao'].replace({'R\$':''}, regex = True)
# df_pichau['preco_boleto'] = df_pichau['preco_boleto'].replace({'R\$':''}, regex = True)
# df_pichau['nome_produto'] = df_pichau['nome_produto'].replace({'\n':''}, regex = True)
# df_pichau.sample(10)

In [15]:
# df_pichau.to_sql('graphic_card', con=engine, if_exists='append', index = False) 

_______________________________________________________

In [16]:
# containers = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
# https://www.kabum.com.br/cgi-local/site/listagem/listagem.cgi?string=processador&btnG=&pagina=1&ordem=5&limite=30&prime=false&marcas=[%225%22,%226%22]&tipo_produto=[]&filtro=[]

In [17]:
# images = html.findAll('div', {"class":"sc-fznKkj fEpBwY"})

# # images.img['src']
# for image in images:
#     print(image.img['src'])
#     # if "indisponivel" in image.img['src']:
#     #     print('Indisponivel')
#     # elif "disponivel" in image.img['src']:
#     #     print('Disponivel')

In [18]:
# preco_antigo_cartao = float((container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2].replace('.', '').replace(',', '.'))
# preco_atual_cartao =  float((container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1].replace('.', '').replace(',', '.'))
# preco_boleto = float((container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1].replace('.', '').replace(',', '.'))


In [19]:
# print(preco_antigo_cartao)
# # print(re.search('^(([1-9]\d{0,2}(\.\d{3})*)|(([1-9]\.\d*)?\d))(\,\d\d)?', preco_atual_cartao).group())
# print(preco_atual_cartao)
# print(preco_boleto)

In [20]:
# item_aux.find('span')
# nao_disponivel
# if "Indisponivel" not in item_aux.find('span').text:
#     print('Entrou aqui')
#     item_aux.find('a', {"class":"product-item-link"}).text
#     item_aux.find('span', {"class":"price"}).text
#     item_aux.find('span', {"class":"price-boleto"}).text.split(" ")[2]
# else:
#     print('Não ta')
# if item_aux.find('div', {"class":"price"}) != None:
#     print('Não ta')
# else:
#     print('Não ta2')